In [ ]:
# this is the final model that blends the outputs from 3 models which come from 3 further models
# it all starts with a notebook on creating the folds https://www.kaggle.com/code/andrewnuk/kfolds-5
# then hypertunes 3 models https://www.kaggle.com/code/andrewnuk/hypertuning-xgb-cb-lgbm
# then runs the 3 models https://www.kaggle.com/code/andrewnuk/models-level1-stacking
# these outputs are hypertuned again https://www.kaggle.com/code/andrewnuk/hypertuning-level2
# and the 3 models then run again https://www.kaggle.com/code/andrewnuk/models-level2
# finally the outputs blended in this notebook https://www.kaggle.com/code/andrewnuk/model-level3

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.linear_model import LinearRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# check on the panda version and its dependencies
# i run this from time to time to ensure all is up to date
pd.__version__
#pd.show_versions()

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
# detect number of folds
fold_no = df_train['kfolds'].max() +1
fold_no

In [ ]:
# output files from https://www.kaggle.com/code/andrewnuk/models-level2/notebook

df1 = pd.read_csv("../input/models-level2/level1_train_pred_1_20220524.csv")
df2 = pd.read_csv("../input/models-level2/level1_train_pred_2_20220524.csv")
df3 = pd.read_csv("../input/models-level2/level1_train_pred_3_20220524.csv")

df_test1 = pd.read_csv("../input/models-level2/level1_test_pred_1_20220524.csv")
df_test2 = pd.read_csv("../input/models-level2/level1_test_pred_2_20220524.csv")
df_test3 = pd.read_csv("../input/models-level2/level1_test_pred_3_20220524.csv")

df_train = df_train.merge(df1, on="id", how="left")
df_train = df_train.merge(df2, on="id", how="left")
df_train = df_train.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

In [ ]:
df_train.head()

In [ ]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

In [ ]:
# final_predictions = []
# final_valid_predictions = {}
# scores = []

# for fold in range(fold_no):
#     xtrain =  df_train[df_train['kfolds'] != fold].reset_index(drop=True)
#     xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)
#     xtest = df_test.copy()
    
#     valid_ids = xvalid.id.values.tolist()

#     ytrain = xtrain.target
#     yvalid = xvalid.target
    
#     xtrain = xtrain[useful_features]
#     xvalid = xvalid[useful_features]
    
# #     ordinal_encoder = preprocessing.OrdinalEncoder()
# #     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
# #     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
# #     xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
# #     model = XGBRegressor(random_state=0, n_jobs=6) # i have 8 cores but want to keep 2 open
    
#     model = XGBRegressor(random_state=0, n_jobs=-1, **params)
       
#     model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
#     preds_valid = model.predict(xvalid)
#     test_preds = model.predict(xtest)
#     final_predictions.append(test_preds)
#     final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
#     #rmse = mean_squared_error(yvalid, preds_valid, squared=False)
#     roc = roc_auc_score(yvalid, preds_valid)
#     print(fold, roc)
#     scores.append(roc)

# print(np.mean(scores), np.std(scores))

In [ ]:
final_predictions = []
final_valid_predictions = {}
scores = []

for fold in range(fold_no):
    xtrain =  df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
#     xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
#     model = XGBRegressor(random_state=0, n_jobs=6) # i have 8 cores but want to keep 2 open
    
    model = LinearRegression()
       
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    #rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    roc = roc_auc_score(yvalid, preds_valid)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores), np.std(scores))

In [ ]:
df_sampleSubmission.target = np.mean(np.column_stack(final_predictions), axis=1)
df_sampleSubmission.to_csv("submission.csv", index=False)